<a href="https://colab.research.google.com/github/coraldx5/generativeai_intro_book/blob/master/chap05_LLM_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第5章：言語モデルを動かしてみよう
- ① 穴埋め問題を解くMLM
- ② 次のトークンを予測するCLM

## MLM（Masked Language Model）を動かてみる
- 穴埋め問題を解いてみましょう

### 穴埋め問題を解く準備

In [1]:
!pip install fugashi==1.3.2 ipadic==1.0.0
import transformers, torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM, pipeline

# トークナイザと訓練済みモデルの読み込み
# 'cl-tohoku/bert-base-japanese-whole-word-masking' という事前学習済みの日本語BERTモデルを使用します。
model = BertForMaskedLM.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# 事前学習済みモデルに対応するトークナイザーをロードします。
# BertJapaneseTokenizerはテキストをトークン（モデルが理解できる単位）に変換し、逆にトークンからテキストに変換する役割を持ちます。
tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# パイプラインの定義
# 'fill-mask' タスクのパイプラインを作成します。これは文章の中の [MASK] トークンを予測するためのものです。
fill_mask = pipeline('fill-mask',
                     model=model,        # 言語モデルの指定
                     tokenizer=tokenizer, # トークナイザの指定
                     top_k=6              # 表示する候補数の指定
                    )

# 結果を表示する関数の定義
# 文章を入力として、[MASK] トークンの候補とその確率を表示する関数を定義します。
def predictmask(text):
    print('---' * 10)
    print(f'元の文章：「{text}」')
    print(f'[MASK]部の候補：')
    for res in fill_mask(text):
        print(f"{res['score']:.4f}: {res['token_str']}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 7.3 MB/s eta 0:00:00
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556704 sha256=22a4ebb7da1cf2cbff8f8a259b17c4d2fa5edc8f574bc5962075fbb2adf3619f
  Stored in directory: /root/.cache/pip/wheels/5b/ea/e3/2f6e0860a327daba3b030853fce4483ed37468bbf1101c59c3
Successfully built ipadic


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

### 穴埋め問題を解く

In [2]:
predictmask('サングラスをかけた[MASK]が公園を駆け回る。')
predictmask('サングラスをかけた[MASK]を食べるのが楽しみだ')
predictmask('生卵をかけた[MASK]を食べるのが楽しみだ')

------------------------------
元の文章：「サングラスをかけた[MASK]が公園を駆け回る。」
[MASK]部の候補：
0.2395: 男
0.0848: 少年
0.0511: 少女
0.0433: 犬
0.0331: 青年
0.0293: 女性
------------------------------
元の文章：「サングラスをかけた[MASK]を食べるのが楽しみだ」
[MASK]部の候補：
0.0882: もの
0.0608: ケーキ
0.0493: 料理
0.0314: 朝食
0.0312: 犬
0.0305: ご飯
------------------------------
元の文章：「生卵をかけた[MASK]を食べるのが楽しみだ」
[MASK]部の候補：
0.1659: もの
0.1010: ご飯
0.0815: 料理
0.0752: スープ
0.0658: 卵
0.0490: パン


In [3]:
# @title ## 参考UI：穴埋め問題を解くサンプルコード
# @markdown 入力欄に、[MASK]を含む文章を入力してください。
check_sentence = "サングラスをかけた[MASK]が公園を駆け回る。" # @param {type:"string"}
predictmask(check_sentence)

------------------------------
元の文章：「サングラスをかけた[MASK]が公園を駆け回る。」
[MASK]部の候補：
0.2395: 男
0.0848: 少年
0.0511: 少女
0.0433: 犬
0.0331: 青年
0.0293: 女性


# CLM（Causal Language Model）を動かしてみよう
- 次のトークンを予測してみよう

### 文章生成の準備

In [7]:
!pip install transformers==4.42.4 torch==2.3.1+cu121
import transformers, torch
from transformers import GPT2LMHeadModel, T5Tokenizer

# モデルとトークナイザーのロード
# 'rinna/japanese-gpt2-medium' という事前学習済みの日本語GPT-2モデルを使用します。モデルのサイズは約1.37GBです。
model_name = 'rinna/japanese-gpt2-medium'  # モデルの名前を指定

# 事前学習済みのGPT-2モデルをロードします。GPT2LMHeadModelはテキスト生成タスクに使用されるモデルです。
model = GPT2LMHeadModel.from_pretrained(model_name)

# 事前学習済みモデルに対応するトークナイザーをロードします。
# T5Tokenizerはテキストをトークン（モデルが理解できる単位）に変換し、逆にトークンからテキストに変換する役割を持ちます。
tokenizer = T5Tokenizer.from_pretrained(model_name)

def generate_text(input_text, max_length):
    # 入力文章をトークン化
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Attention maskの設定
    attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

    # テキスト生成
    output = model.generate(
        input_ids,
        attention_mask=attention_mask,  # Attention maskを指定
        max_length=max_length,  # 生成する最大トークン数
        # no_repeat_ngram_size=2,  # 繰り返しを防ぐn-gramのサイズ
        pad_token_id=tokenizer.pad_token_id,  # パディングのトークンID
        bos_token_id=tokenizer.bos_token_id,  # テキスト先頭のトークンID
        eos_token_id=tokenizer.eos_token_id,  # テキスト終端のトークンID
    )

    # 生成されたテキストのデコード
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_text

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [8]:
# 関数の使用例
input_text = "これから雨が降りそうなので、"  # 入力テキスト
max_length = 50  # 生成する最大トークン数

# 生成されたテキストを表示
generated_text = generate_text(input_text, max_length)
print(generated_text)

これから雨が降りそうなので、今日は、お休みです。 明日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日


In [ ]:
# @title ## 参考UI：後続文章を生成するサンプルコード
# @markdown 文章を入力して下さい
input_text = "これから雨が降りそうなので、" # @param {type:"string"}
# @markdown 生成する最大トークン数
max_length = 50 # @param {type:"integer"}
generate_text(input_text, max_length)

'これから雨が降りそうなので、今日は、お休みです。 明日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日'

## 各トークンの予測確率を可視化してみよう

### 貪欲探索の挙動を確認する

In [10]:
# モデルを評価モードに設定
model.eval()

# 入力文章をトークン化
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Attention maskの設定
attention_mask = torch.ones(input_ids.shape, dtype=torch.long)

# 生成されたトークンとその確率を順次表示
max_length = 3  # 生成する最大トークン数
for _ in range(max_length):
    # トークンの予測確率を取得
    with torch.no_grad():
        outputs = model(input_ids)
        predictions = outputs.logits

    # 次のトークンの予測確率を計算
    next_token_probs = torch.softmax(predictions[:, -1, :], dim=-1)

    # 上位3つのトークンを取得
    top_k = 3
    top_k_probs, top_k_indices = torch.topk(next_token_probs, top_k)

    # 上位3つのトークンとその確率を表示
    print(f"\n({_+1}番目) 上位3つのトークンと、確率：")
    for i in range(top_k):
        predicted_token_id = top_k_indices[0, i].item()
        predicted_token = tokenizer.decode([predicted_token_id])
        predicted_prob = top_k_probs[0, i].item()
        print(f"Token: {predicted_token}({predicted_token_id}), Probability: {predicted_prob:.4f}")

    # 最も確率の高いトークンを入力トークンに追加
    input_ids = torch.cat((input_ids, top_k_indices[:, 0].unsqueeze(-1)), dim=1)

    # 予測が終了トークンに到達した場合は終了
    if top_k_indices[0, 0].item() == tokenizer.eos_token_id:
        break

# 生成された全テキストを表示
generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
print("---"*10)
print("生成された文章:")
print(generated_text)


(1番目) 上位3つのトークンと、確率：
Token: 今日(4761), Probability: 0.0668
Token: 雨(1537), Probability: 0.0178
Token: 早(1745), Probability: 0.0128

(2番目) 上位3つのトークンと、確率：
Token: は(11), Probability: 0.7620
Token: も(30), Probability: 0.0637
Token: はこの(1007), Probability: 0.0170

(3番目) 上位3つのトークンと、確率：
Token: 、(7), Probability: 0.1085
Token: (9), Probability: 0.0632
Token: お(220), Probability: 0.0614
------------------------------
生成された文章:
これから雨が降りそうなので、今日は、


### 様々な探索手法を体験する

In [ ]:
import transformers, torch
from transformers import GPT2LMHeadModel, T5Tokenizer

# モデルとトークナイザーのロード
model_name = 'rinna/japanese-gpt2-medium'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

# テキスト生成の設定
input_text = "これから雨が降りそうなので、"  # 入力テキスト
max_length = 40  # 生成する最大トークン数

# トークン化
# 入力テキストをモデルが理解できるトークンに変換します
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# テキスト生成のパラメータを辞書で設定
# max_length: 生成する最大トークン数
# pad_token_id: パディングのトークンID
# bos_token_id: テキスト先頭のトークンID
# eos_token_id: テキスト終端のトークンID
prm = {
    "max_length": max_length,
    "pad_token_id": tokenizer.pad_token_id,
    "bos_token_id": tokenizer.bos_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}

# Greedy探索
# Greedy探索は、各ステップで最も確率の高いトークンを選びます
greedy_output = model.generate(input_ids, **prm)
print("Greedy:", tokenizer.decode(greedy_output[0], skip_special_tokens=True))

# Beam探索
# Beam探索は、複数の候補（ビーム）を同時に探索し、最も良い結果を選びます
# num_beams: ビームの数
# early_stopping: 生成の早期終了を行うかどうか
beam_output = model.generate(input_ids, num_beams=3, early_stopping=True, **prm)
print("Beam:", tokenizer.decode(beam_output[0], skip_special_tokens=True))

# Top-kサンプリング
# Top-kサンプリングは、上位k個のトークンからランダムに選択します
# do_sample: サンプリングを行うかどうか
# top_k: 選択する上位トークンの数
top_k_output = model.generate(input_ids, do_sample=True, top_k=50, **prm)
print("Top-k Sampling:", tokenizer.decode(top_k_output[0], skip_special_tokens=True))

# Top-pサンプリング
# Top-pサンプリングは、確率の高いトークンの集合からランダムに選択します
# do_sample: サンプリングを行うかどうか
# top_p: 累積確率がpを超えるまでのトークンを選択する閾値
top_p_output = model.generate(input_ids, do_sample=True, top_p=0.95, **prm)
print("Top-p Sampling:", tokenizer.decode(top_p_output[0], skip_special_tokens=True))


Greedy: これから雨が降りそうなので、今日は、お休みです。 明日は、お休みです。 今日は、お休みです。 今日は、お休みです。
Beam: これから雨が降りそうなので、早めに切り上げました。 今日は、朝から雨が降っていましたが、 午後からは晴れてきました。 今日は
Top-k Sampling: これから雨が降りそうなので、今日はこの程度でおしまい。 皆さま、お仕事頑張ってください。 さて、先週の水曜日に開催された、お台場冒険王
Top-p Sampling: これから雨が降りそうなので、洗濯物を外に出したいと思っています。 今日は暖かくなるみたいですね 今日は暖かいので 春に向けて ヘアアレンジしちゃ


In [ ]:
# @title ## 参考UI：様々な探索法で文章を生成するサンプルコード
# @markdown 文章を入力して下さい
# テキスト生成の設定
input_text = "これから雨が降りそうなので、" # @param {type:"string"}
# @markdown 生成する最大トークン数
max_length = 50 # @param {type:"integer"}

# トークン化
# 入力テキストをモデルが理解できるトークンに変換します
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# テキスト生成のパラメータを辞書で設定
# max_length: 生成する最大トークン数
# pad_token_id: パディングのトークンID
# bos_token_id: テキスト先頭のトークンID
# eos_token_id: テキスト終端のトークンID
prm = {
    "max_length": max_length,
    "pad_token_id": tokenizer.pad_token_id,
    "bos_token_id": tokenizer.bos_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}

# Greedy探索
# Greedy探索は、各ステップで最も確率の高いトークンを選びます
greedy_output = model.generate(input_ids, **prm)
print("Greedy:", tokenizer.decode(greedy_output[0], skip_special_tokens=True))

# Beam探索
# Beam探索は、複数の候補（ビーム）を同時に探索し、最も良い結果を選びます
# num_beams: ビームの数
# early_stopping: 生成の早期終了を行うかどうか
beam_output = model.generate(input_ids, num_beams=3, early_stopping=True, **prm)
print("Beam:", tokenizer.decode(beam_output[0], skip_special_tokens=True))

# Top-kサンプリング
# Top-kサンプリングは、上位k個のトークンからランダムに選択します
# do_sample: サンプリングを行うかどうか
# top_k: 選択する上位トークンの数
top_k_output = model.generate(input_ids, do_sample=True, top_k=50, **prm)
print("Top-k Sampling:", tokenizer.decode(top_k_output[0], skip_special_tokens=True))

# Top-pサンプリング
# Top-pサンプリングは、確率の高いトークンの集合からランダムに選択します
# do_sample: サンプリングを行うかどうか
# top_p: 累積確率がpを超えるまでのトークンを選択する閾値
top_p_output = model.generate(input_ids, do_sample=True, top_p=0.95, **prm)
print("Top-p Sampling:", tokenizer.decode(top_p_output[0], skip_special_tokens=True))

Greedy: これから雨が降りそうなので、今日は、お休みです。 明日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日は、お休みです。 今日
Beam: これから雨が降りそうなので、早めに切り上げました。 今日は、朝から雨が降っていましたが、 午後からは晴れてきました。 今日は、朝から雨が降っていましたが、
Top-k Sampling: これから雨が降りそうなので、皆様、お気をつけていらして下さい(.. )ノ*.。.:*・ 私もまだまだ、手探り状態ですが、今年も
Top-p Sampling: これから雨が降りそうなので、室内や玄関などには、日陰で 風通しの良いところに干すことをおすすめします。 1番簡単なのは、梅干しを干す時の風通しの良い場所に干す


# 言語モデルをファインチューニングして分類問題を解く

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim

# データセットの作成
class CommentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item


# GitHubからCSVファイルをダウンロードします
!wget https://raw.githubusercontent.com/coraldx5/generativeai_intro_book/master/movie_review_jpn.csv
# データの読み込み
df = pd.read_csv("movie_review_jpn.csv")

# データの確認
print(df.head())

# ラベルのエンコーディング
df['label'] = df['result'].map({'good': 1, 'bad': 0})

# 訓練データとテストデータに分割
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# トークナイザーのロード
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese')

# テキストのトークナイズとエンコード
def encode_data(data):
    return tokenizer.batch_encode_plus(
        data['comment'].tolist(),
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

train_encodings = encode_data(train_df)
test_encodings = encode_data(test_df)

train_labels = torch.tensor(train_df['label'].values)
test_labels = torch.tensor(test_df['label'].values)

train_dataset = CommentDataset(train_encodings, train_labels)
test_dataset = CommentDataset(test_encodings, test_labels)

train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False)

--2024-07-15 11:56:29--  https://raw.githubusercontent.com/coraldx5/generativeai_intro_book/master/movie_review_jpn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12920 (13K) [text/plain]
Saving to: ‘movie_review_jpn.csv’

movie_review_jpn.cs 100%[===================>]  12.62K  --.-KB/s    in 0s      

2024-07-15 11:56:30 (79.0 MB/s) - ‘movie_review_jpn.csv’ saved [12920/12920]

  result                           comment
0   good     キャラクターたちの感情的な旅路が感動的で、心に残りました。
1    bad       キャラクターたちの関係が浅く、深みがありませんでした。
2   good     キャラクターたちの成長が感動的で、彼らの旅に共感しました。
3   good   キャラクターたちの成長と絆が感動的で、深い感情を呼び覚ました。
4    bad  キャラクターたちの成長と絆が不明瞭で、感情が伝わりませんでした。


tokenizer_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/258k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/479 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

# モデルのロード
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese', num_labels=2)
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(1):  #← エポック数（学習量に相当します）
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1} completed. Loss: {loss.item()}")

    model.eval()
    eval_loss = 0
    eval_steps = 0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            eval_loss += loss.item()
            eval_steps += 1

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    avg_eval_loss = eval_loss / eval_steps
    accuracy = correct_predictions / total_predictions
    print(f"Validation Loss: {avg_eval_loss}")
    print(f"Validation Accuracy: {accuracy}")

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 completed. Loss: 0.8773646354675293
Validation Loss: 0.5938543975353241
Validation Accuracy: 0.8846153846153846


In [ ]:
def predict(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return 'good' if predicted_class == 1 else 'bad'

# 予測の例
new_comment = "場面転換が唐突で、ストーリーの流れが断片的に感じられました。"
prediction = predict(new_comment)
print(f"The predicted sentiment for the comment is: {prediction}")

The predicted sentiment for the comment is: bad


In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# モデル名を設定（Helsinki-NLPの日本語-英語モデルを使用）
model_name = 'Helsinki-NLP/opus-mt-ja-en'

# トークナイザーとモデルをロード
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 翻訳する日本語の文章
input_sentence = "空は青く雲は白い"

# トークン化
input_tokens = tokenizer(input_sentence, return_tensors='pt')

# 翻訳の生成
translated_tokens = model.generate(**input_tokens)

# トークンをデコードして英語の文章に変換
translated_sentence = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)

print(translated_sentence)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/782k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/303M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

The sky is blue and the clouds are white.
